# <B> Setup for Anormaly Detection with SageMaker </B>
* Container: codna_python3

## AutoReload

In [ ]:
%load_ext autoreload
%autoreload 2

## 0. Install packages

In [ ]:
install_needed = True  # should only be True once
# install_needed = False

In [ ]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

In [ ]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U awswrangler #==2.20.1
    !{sys.executable} -m pip install -U sagemaker
    !{sys.executable} -m pip install -U datasets transformers s3fs
    !sudo curl -L "https://github.com/docker/compose/releases/download/v2.26.1/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
    !sudo chmod +x /usr/local/bin/docker-compose
        
    IPython.Application.instance().kernel.do_shutdown(True)

## 1. Set default parameters

In [ ]:
import boto3
import sagemaker

### Bucket / Prefix 설정

In [ ]:
strRegionName = boto3.Session().region_name
strAccountId = boto3.client("sts").get_caller_identity().get("Account")
strBucketName = 'sm-anomaly-detection-dongjin' # <-- 사용할 bucket 명을 추가해 주세요. ex) sagemaker-us-east-1-123456789123, sm-nemo-bucket
strPrefix = 'ad-ts' ## <-- 작업할 prefix 명을 추가해 주세요. ex) nemo-test, nemo-asr

## 2. Extract & Upload dataset

In [ ]:
import os
from utils.s3 import s3_handler

In [ ]:
strLocalDataPath = "./data"
!tar -zxvf $strLocalDataPath/data.tar.gz --directory=data

* create bucket

In [ ]:
s3 = s3_handler(strRegionName)

In [ ]:
import boto3
s3_client = boto3.client('s3')
s3_client.create_bucket(Bucket=strBucketName)

* updoad data to s3

In [ ]:
source_dir, target_bucket, target_dir = strLocalDataPath, strBucketName, "data"
s3.upload_dir(source_dir, target_bucket, target_dir)

## 3. 파라미터 저장/활용

In [ ]:
# 변수 저장
%store strPrefix
%store strRegionName
%store strBucketName
%store strAccountId